In [2]:
import os
import shutil
import pandas as pd
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# Paths configuration

# PATH_ROOT = "/media/tom-ratsakatika/CRUCIAL 4TB/FCC Camera Trap Data"
PATH_ROOT = "/home/tom-ratsakatika/VSCode/camera-traps/data/"

labelled_data_path = '../data/manual_labelling/fcc_spreadsheet_df_labels_ground_truth.xlsx'

df = pd.read_excel(labelled_data_path, sheet_name='structured_data')

df.head()

,Sort,Session,Category,GMU,TrapSite,Longitude,Latitude,Altitude,StartDate,EndDate,...,score,count,humanpresence,FCC_Master_Class,DF(base)_Master_Class,DF(sequence)_Master_Class,FCC_DF(base)_Match,FCC_DF(sequence)_Match,DF(base)_DF(sequence)_Match,Ground_Truth
0,9,1,Systematic monitoring of lynx,34SercaitaBV,Ohaba,25.161698,45.732235,600,2018-02-08,2018-04-09,...,0.97,1,0,Fox,Fox,Fox,True,True,True,Fox
1,9,1,Systematic monitoring of lynx,34SercaitaBV,Ohaba,25.161698,45.732235,600,2018-02-08,2018-04-09,...,0.97,1,0,Fox,Fox,Fox,True,True,True,Fox
2,10,1,Systematic monitoring of lynx,34SercaitaBV,Ohaba,25.161698,45.732235,600,2018-02-08,2018-04-09,...,1.00,0,0,Unknown,Empty**,Empty**,False,False,True,Unknown
3,11,1,Systematic monitoring of lynx,34SercaitaBV,Ohaba,25.161698,45.732235,600,2018-02-08,2018-04-09,...,0.97,0,0,Fox,Empty**,Fox,False,True,False,Fox
4,11,1,Systematic monitoring of lynx,34SercaitaBV,Ohaba,25.161698,45.732235,600,2018-02-08,2018-04-09,...,0.97,1,0,Fox,Fox,Fox,True,True,True,Fox


In [4]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

# Define a list of animals to analyze
animals = ['Wild boar', 'Bear', 'Lynx', 'Wolf']

def adjust_classes(data, animal):
    data = data.copy()
    data['Adjusted Ground Truth'] = data['Ground_Truth'].apply(lambda x: animal if x == animal else 'Not ' + animal)
    data['Adjusted Prediction'] = data['DF(sequence)_Master_Class'].apply(lambda x: animal if x == animal else 'Not ' + animal)
    return data

def calculate_metrics(data):
    if data.empty:
        return (np.nan, np.nan, np.nan, 0)  # Return NaN for metrics and 0 for count if no data to process
    y_true = data['Adjusted Ground Truth']
    y_pred = data['Adjusted Prediction']
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', pos_label=data['Adjusted Ground Truth'].unique()[0], zero_division=0)
    n = len(data)
    return precision, recall, f1, n

# Conditions based on camera type and other criteria
conditions = {
    "RGB Cameras (Day)": (df['Camera Brand'] == 'CUDDEBACK') & (df['Flash Used'] == 'N'),
    "RGB Cameras (Night)": (df['Camera Brand'] == 'CUDDEBACK') & (df['Flash Used'] == 'Y'),
    "IR Cameras (Day)": (df['Camera Brand'] != 'CUDDEBACK') & (df['Day/Night'] == 'Day'),
    "IR Cameras (Night)": (df['Camera Brand'] != 'CUDDEBACK') & (df['Day/Night'] == 'Night'),
    "All Cameras (Day)": ((df['Camera Brand'] == 'CUDDEBACK') & (df['Flash Used'] == 'N')) | ((df['Camera Brand'] != 'CUDDEBACK') & (df['Day/Night'] == 'Day')),
    "All Cameras (Night)": ((df['Camera Brand'] == 'CUDDEBACK') & (df['Flash Used'] == 'Y')) | ((df['Camera Brand'] != 'CUDDEBACK') & (df['Day/Night'] == 'Night'))
}

# Compute and display metrics for each condition for each animal
stats_table = []
for animal in animals:
    for condition, mask in conditions.items():
        filtered_data = df[mask & (df['Ground_Truth'] == animal)]
        adjusted_data = adjust_classes(filtered_data, animal)
        precision, recall, f1, n = calculate_metrics(adjusted_data)
        stats_table.append({
            'Animal': animal,
            'Condition': condition,
            'n': n,
            'Precision': f"{precision:.3f}",
            'Recall': f"{recall:.3f}",
            'F1 Score': f"{f1:.3f}",
        })

# Convert to DataFrame for nicer display
stats_df = pd.DataFrame(stats_table)
stats_df


,Animal,Condition,n,Precision,Recall,F1 Score
0,Wild boar,RGB Cameras (Day),634,1.000,0.983,0.991
1,Wild boar,RGB Cameras (Night),829,1.000,0.943,0.971
2,Wild boar,IR Cameras (Day),497,1.000,0.990,0.995
3,Wild boar,IR Cameras (Night),1333,1.000,0.931,0.964
4,Wild boar,All Cameras (Day),1131,1.000,0.986,0.993
5,Wild boar,All Cameras (Night),2162,1.000,0.936,0.967
6,Bear,RGB Cameras (Day),790,1.000,0.948,0.973
7,Bear,RGB Cameras (Night),594,1.000,0.963,0.981
8,Bear,IR Cameras (Day),639,1.000,0.975,0.987
9,Bear,IR Cameras (Night),913,1.000,0.947,0.973


In [6]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

def adjust_classes(data):
    data = data.copy()
    data['Adjusted Ground Truth'] = data['Ground_Truth'].apply(lambda x: 'Animal' if x in animals else 'Not Animal')
    data['Adjusted Prediction'] = data['DF(sequence)_Master_Class'].apply(lambda x: 'Animal' if x in animals else 'Not Animal')
    return data

def calculate_metrics(data):
    if data.empty:
        return (np.nan, np.nan, np.nan, 0)  # Return NaN for metrics and 0 for count if no data to process
    y_true = data['Adjusted Ground Truth']
    y_pred = data['Adjusted Prediction']
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', pos_label='Animal', zero_division=0)
    n = len(data)
    return precision, recall, f1, n

# Conditions based on camera type and other criteria
conditions = {
    "RGB Cameras (Day)": (df['Camera Brand'] == 'CUDDEBACK') & (df['Flash Used'] == 'N'),
    "RGB Cameras (Night)": (df['Camera Brand'] == 'CUDDEBACK') & (df['Flash Used'] == 'Y'),
    "IR Cameras (Day)": (df['Camera Brand'] != 'CUDDEBACK') & (df['Day/Night'] == 'Day'),
    "IR Cameras (Night)": (df['Camera Brand'] != 'CUDDEBACK') & (df['Day/Night'] == 'Night'),
    "All Cameras (Day)": ((df['Camera Brand'] == 'CUDDEBACK') & (df['Flash Used'] == 'N')) | ((df['Camera Brand'] != 'CUDDEBACK') & (df['Day/Night'] == 'Day')),
    "All Cameras (Night)": ((df['Camera Brand'] == 'CUDDEBACK') & (df['Flash Used'] == 'Y')) | ((df['Camera Brand'] != 'CUDDEBACK') & (df['Day/Night'] == 'Night'))
}

# Compute and display metrics for each condition
stats_table = []
for condition, mask in conditions.items():
    filtered_data = df[mask]
    adjusted_data = adjust_classes(filtered_data)
    precision, recall, f1, n = calculate_metrics(adjusted_data)
    stats_table.append({
        'Condition': condition,
        'n': n,
        'Precision': f"{precision:.4f}",
        'Recall': f"{recall:.4f}",
        'F1 Score': f"{f1:.4f}",
    })

# Convert to DataFrame for nicer display
stats_df = pd.DataFrame(stats_table)
stats_df


,Condition,n,Precision,Recall,F1 Score
0,RGB Cameras (Day),5893,0.9989,0.9676,0.9830
1,RGB Cameras (Night),4130,0.9949,0.9506,0.9722
2,IR Cameras (Day),4168,0.9963,0.9813,0.9888
3,IR Cameras (Night),5406,0.9954,0.9082,0.9498
4,All Cameras (Day),10061,0.9978,0.9734,0.9854
5,All Cameras (Night),9536,0.9952,0.9247,0.9587
